In [5]:
import pandas as pd
import numpy as np

In [ ]:

df = pd.read_csv(r'C:\hanchon\data\processed\final_integrated_dataset.csv')

# 가장 많이 팔린 메뉴명 상위 100개 출력
print(df['메뉴명'].value_counts().head(100))

메뉴명
설렁탕          68256
얼큰설렁탕        59988
공기밥          44882
양곰탕          43733
UNKNOWN      43694
             ...  
배)소머리곰탕       6610
접시수육          6407
맛수육           6404
배)갈비만두(大)     6331
설렁탕집 육개장      6308
Name: count, Length: 100, dtype: int64


In [3]:
# '세트' 또는 '+'가 포함된 메뉴명 탐색
set_menu_keywords = '세트|\+'
set_menus = df[df['메뉴명'].str.contains(set_menu_keywords, na=False)]

print(f"\n--- 세트/조합 메뉴 (총 {len(set_menus)}건) ---")
print(set_menus['메뉴명'].value_counts().head(20))

# '추가'가 포함된 메뉴명 탐색
side_menu_keywords = '추가|사리'
side_menus = df[df['메뉴명'].str.contains(side_menu_keywords, na=False)]

print(f"\n--- 추가/사이드 메뉴 (총 {len(side_menus)}건) ---")
print(side_menus['메뉴명'].value_counts().head(20))


--- 세트/조합 메뉴 (총 169835건) ---
메뉴명
포장육수1+1             16898
냉장육수 1+1             8094
설렁탕 + 수만두2알          7388
얼큰설렁탕 + 수만두2알        5201
배)설렁탕 + 수만두2알        4710
설렁탕 + 갈비만두3알         4609
배)설렁탕 + 갈비만두3알       4546
배)1인 설렁탕+갈비만두3       4364
배)1인 얼큰설렁탕+수만두2      4220
배)1인 설렁탕+수만두2        3325
배)얼큰설렁탕 + 수만두2알      3277
물냉+수만두2알             3191
얼큰설렁탕 + 갈비만두3알       2976
배)냉장육수 1+1           2940
1인 설렁탕+갈비만두3         2688
물냉 + 수만두2알           2632
물냉+갈비만두3알            2542
배)얼큰설렁탕 + 갈비만두3알     2541
비냉+수만두2알             2354
비냉 + 수만두2알           2072
Name: count, dtype: int64

--- 추가/사이드 메뉴 (총 80097건) ---
메뉴명
배)사리추가            18958
배)김치추가            11028
(배달)김치추가           9288
설렁탕(사리빼고)          3236
배)육수추가             2697
국수사리 추가            2618
설렁탕(사리빼기)          2494
공기밥 추가             2295
김치추가               2076
물냉사리               1517
+ 국수사리 추가          1435
사리추가 1,000         1153
비냉사리               1001
배)당면추가              901
쿠)사리추가              879
콜라 추가               857
사리

In [ ]:
# 1. 04단계 통합 데이터셋
df = pd.read_csv(r'C:\hanchon\data\processed\final_integrated_dataset.csv')

print("데이터 로드 완료. 피처 엔지니어링 시작...")

# 2. 메뉴명 키워드를 기반으로 새로운 카테고리 피처(0 또는 1)를 생성
#    na=False는 NaN(UNKNOWN)이 키워드에 포함되지 않도록 한다.

# 2a. 비-제품(수수료/행사) 여부
fee_keywords = '배달료|배달팁|배달비|행사|보냉백'
df['is_fee'] = df['메뉴명'].str.contains(fee_keywords, na=False).astype(int)

# 2b. 세트/조합 메뉴 여부
set_keywords = '세트|\+' # '+'는 특수문자라 \ (역슬래시) 필요
df['is_set'] = df['메뉴명'].str.contains(set_keywords, na=False).astype(int)

# 2c. 사이드/추가 메뉴 여부
side_keywords = '추가|공기밥|사리'
df['is_side'] = df['메뉴명'].str.contains(side_keywords, na=False).astype(int)

# 2d. 배달/포장/HMR 전용 제품 여부
delivery_keywords = '배\)|(배달)|포장|냉장|HMR'
df['is_delivery_item'] = df['메뉴명'].str.contains(delivery_keywords, na=False).astype(int)

# 3. 'is_fee'를 활용하여 "제품"에 대해서만 단가를 계산합니다.
condition = (df['is_fee'] == 0) & (df['수량'] > 0)
df['unit_price'] = np.where(
    condition,
    df['매출액'] / df['수량'],
    0  # 수수료, 수량0, UNKNOWN 등은 0원으로 처리
)

print("신규 피처 5개(is_fee, is_set, is_side, is_delivery_item, unit_price) 생성 완료")

# 4. 결과 확인
print("\n--- 수수료 항목(is_fee=1) 확인 ---")
print(df[df['is_fee'] == 1][['메뉴명', 'unit_price']].head())

print("\n--- 세트 항목(is_set=1) 확인 ---")
print(df[df['is_set'] == 1][['메뉴명', 'unit_price']].head())

데이터 로드 완료. 피처 엔지니어링 시작...


C:\Users\leesk\AppData\Local\Temp\ipykernel_79444\3199427257.py:23: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['is_delivery_item'] = df['메뉴명'].str.contains(delivery_keywords, na=False).astype(int)


신규 피처 5개(is_fee, is_set, is_side, is_delivery_item, unit_price) 생성 완료

--- 수수료 항목(is_fee=1) 확인 ---
         메뉴명  unit_price
20       배달료         0.0
43       배달료         0.0
46       배달료         0.0
47       배달료         0.0
99  배달료 1000         0.0

--- 세트 항목(is_set=1) 확인 ---
                메뉴명    unit_price
14          포장육수1+1   6315.000000
19   배)1인 설렁탕+갈비만두3  12152.500000
23          포장육수1+1  10824.636364
83          포장육수1+1   6526.000000
138         포장육수1+1   5406.000000


In [ ]:
# 5. 모델링에 사용할 최종 데이터셋 저장
output_path = r'C:\hanchon\data\processed\model_ready_dataset.csv'
df.to_csv(output_path, index=False)

print(f"\n피처 엔지니어링 완료! 모델 학습용 데이터 저장: {output_path}")
print(df.info())


✅ 피처 엔지니어링 완료! 모델 학습용 데이터 저장: C:\hanchon\data\processed\model_ready_dataset.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2398650 entries, 0 to 2398649
Data columns (total 27 columns):
 #   Column             Dtype  
---  ------             -----  
 0   date               object 
 1   상권                 object 
 2   홀배달여부              object 
 3   매장명                object 
 4   메뉴코드               object 
 5   메뉴명                object 
 6   매출액                float64
 7   수량                 int64  
 8   temp_avg           float64
 9   temp_min           float64
 10  temp_max           float64
 11  precipitation_sum  float64
 12  is_holiday         int64  
 13  is_event           int64  
 14  year               int64  
 15  month              int64  
 16  quarter            int64  
 17  상권지수               float64
 18  매장지수               float64
 19  floating_pop       float64
 20  resident_pop       float64
 21  working_pop        float64
 22  unit_price         float64
 23  i